[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sebascarag/AI-SyntheticSound/blob/main/VGGish%20Urbansound8K.ipynb)

## Descargar dataset Urbansound8K desde Kaggle

In [1]:
!pip install kaggle
import os

!mkdir ~/.kaggle # crear carpeta en root
# Cargar archivos desde el equipo local
if not os.path.exists("/root/.kaggle/kaggle.json"):
  from google.colab import files
  print('Cargar archivo JSON con el token de Kaggle')
  files.upload();
  !cp kaggle.json ~/.kaggle/ #copiar token hacia la carpeta creada
  !chmod 600 ~/.kaggle/kaggle.json # asigna el permiso necesario

if not os.path.exists("/content/urbansound8k"):
  import kaggle
  kaggle.api.authenticate()
  kaggle.api.dataset_download_files('chrisfilo/urbansound8k', path='/content/urbansound8k', unzip=True)

Saving kaggle.json to kaggle.json


## VGGish

In [2]:
!pip install resampy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.5 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torchaudio
import tensorflow_hub as hub
from IPython.display import Audio

In [4]:
full_data = pd.read_csv('/content/urbansound8k/UrbanSound8K.csv')
full_data['duration'] = full_data['end'] - full_data['start']

selected_data = full_data[full_data.duration >= 4]
selected_data.head(10)

,slice_file_name,fsID,start,end,salience,fold,classID,class,duration
1,100263-2-0-117.wav,100263,58.5,62.5,1,5,2,children_playing,4.0
2,100263-2-0-121.wav,100263,60.5,64.5,1,5,2,children_playing,4.0
3,100263-2-0-126.wav,100263,63.0,67.0,1,5,2,children_playing,4.0
4,100263-2-0-137.wav,100263,68.5,72.5,1,5,2,children_playing,4.0
5,100263-2-0-143.wav,100263,71.5,75.5,1,5,2,children_playing,4.0
6,100263-2-0-161.wav,100263,80.5,84.5,1,5,2,children_playing,4.0
7,100263-2-0-3.wav,100263,1.5,5.5,1,5,2,children_playing,4.0
8,100263-2-0-36.wav,100263,18.0,22.0,1,5,2,children_playing,4.0
14,100652-3-0-0.wav,100652,0.0,4.0,1,2,3,dog_bark,4.0
15,100652-3-0-1.wav,100652,0.5,4.5,1,2,3,dog_bark,4.0


In [5]:
# Cargar modelo y pasar a modo evaluación:
model = torch.hub.load('harritaylor/torchvggish', 'vggish')
model.eval()

/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/harritaylor/torchvggish/zipball/master" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/harritaylor/torchvggish/releases/download/v0.1/vggish-10086976.pth" to /root/.cache/torch/hub/checkpoints/vggish-10086976.pth
100%|██████████| 275M/275M [00:01<00:00, 239MB/s]
Downloading: "https://github.com/ha

VGGish(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False

In [6]:
selected_data.shape[0]

7077

In [11]:
extracted_features = torch.empty((0, 4, 128)) # 0= no se tiene definido hasta el momento el tamaño
labels = np.empty(0)

for idx, row in tqdm(selected_data.iterrows(), total=selected_data.shape[0]):
  path_file = f"/content/urbansound8k/fold{row['fold']}/{row['slice_file_name']}"
  vggish_emb = model.forward(path_file)
  extracted_features = torch.cat((extracted_features, vggish_emb.detach().cpu().view(1, 4, 128)), axis=0)
  labels = np.append(labels, row["classID"])


Streaming output truncated to the last 5000 lines.
100%|██████████| 7077/7077 [22:12<00:00,  5.31it/s]


In [12]:
extracted_features.shape

torch.Size([7077, 4, 128])

In [13]:
labels.shape

(7077,)

In [14]:
extracted_features_fl = torch.flatten(extracted_features, start_dim=1).numpy()
extracted_features_fl = np.append(extracted_features_fl, labels[:,None], axis=1)

In [15]:
extracted_features_fl.shape

(7077, 513)

In [16]:
import h5py
# with: context manager
with h5py.File("vggish_features_labels.hdf5", "w") as f:
    dset = f.create_dataset("data", extracted_features_fl.shape, data=extracted_features_fl)